In [ ]:
# IMPORTS


import string
import random
import os

# general 
import pandas as pd
import numpy as np
import re
import sklearn 
import string
import random
import os
import spacy

# plotting
import seaborn as sns
import matplotlib.pyplot as plt

# gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
#from gensim.models.wrappers import LdaMallet

# NLTK
import nltk
import nltk.corpus
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

from collections import Counter
from wordcloud import WordCloud

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

%matplotlib inline

In [ ]:
df = pd.read_csv("path to/horoscopedotcom.csv", sep=',')

In [ ]:
horoscope = []
date = []
for line in range(len(df)):
    tmp = df["Horoscope"][line].split('-')
    date.append(tmp[0])
    horoscope.append(' '.join(tmp[1:]))

df['Horoscope'] = horoscope
df['Date'] = date
df.head()

In [ ]:
df = df[df["Date"].str.contains("Feb 4, 2023") == False] #Remove horoscopes of Feb 4, 2023
df=df.reset_index()
df


In [ ]:
df["Sign"].value_counts() #331 horoscopes in total

In [ ]:
df["Date"].value_counts()

In [ ]:
df["Horoscope"].value_counts() #two horoscopes are the same

In [ ]:
df.drop_duplicates(subset = ['Horoscope', 'Sign'], keep='first', inplace=True)
df.shape #1 duplicate dropped

In [ ]:
# count toal number of characters and mean length of an horoscope
count = df['Horoscope'].str.split().str.len()
count.index = count.index.astype(str) + ' words:'
count.sort_index(inplace=True)

In [ ]:
print('Total number of words:', count.sum(), 'words')
print('Mean number of words per horoscope:', round(count.mean(), 2), 'words')

df['horoscope_length'] = df['Horoscope'].str.len()
print('Total length of dataset is:', df.horoscope_length.sum(), 'characters')

print('Mean Length of an horoscope is:', round(df.horoscope_length.mean(), 0), 'characters')
df = df.drop(['horoscope_length'], axis=1)

In [ ]:
#Remove non-letters
df['tidy_horoscope'] = df['Horoscope'].str.replace('[^a-zA-Z#]', ' ') # [^a-zA-Z#] --> non letter

#lower the text 
df['tidy_horoscope'] = df['tidy_horoscope'].str.lower()

#remove words with length =1 
df['tidy_horoscope'] = df['tidy_horoscope'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))

In [ ]:
# Remove Stopwords and custom stopwords
# prepare stopwords
stop_words = stopwords.words('english')
stop_words.extend(['aries', 'taurus', 'gemini', 'cancer', 'leo', 'virgo', 'libra', 'scorpio', 'saggitarius','sagittarius', 'capricorn', 'acquarius','pisces', 'aquarius', 'today'])

In [ ]:
def remove_stopwords(horoscopes):
    return [[word for word in simple_preprocess(str(horoscope)) if word not in stop_words] for horoscope in horoscopes]

In [ ]:

df['tidy_horoscope'] = remove_stopwords(df['tidy_horoscope'])

Tokenization

In [ ]:
# importing word_tokenize from nltk
from nltk.tokenize import word_tokenize


In [ ]:
df['horoscope_token'] = df['tidy_horoscope']
df.head()

In [ ]:
# finding the frequency distinct in the tokens
# Importing FreqDist library from nltk and passing token into FreqDist
from nltk.probability import FreqDist

words=[]
for oroscopo in df.index:
    for parola in range(len(df['horoscope_token'][oroscopo])):
        words.append(df['horoscope_token'][oroscopo][parola])


fdist = FreqDist(words)
fdist


In [ ]:
# To find the frequency of top 10 words
fdist1 = fdist.most_common(10)
fdist1

In [ ]:
df.head()

Stemming: normalizing words into its base form or root form

In [ ]:
# Importing Porterstemmer from nltk library

from nltk.stem import PorterStemmer
pst = PorterStemmer()

horoscope_stemming = []
for riga in df.index:
    horoscope_stemming_tmp= []
    for i in range(len(df['horoscope_token'][riga])):
        tmp = pst.stem(df['horoscope_token'][riga][i])
        horoscope_stemming_tmp.append(tmp)
    horoscope_stemming.append(horoscope_stemming_tmp)

df['horoscope_stemming'] = horoscope_stemming
df.head()
#df['stemmed'] = df['lemmatized'].apply(lambda x: [stemmer.stem(y) for y in x])

Lemmatization

In [ ]:
# Importing Lemmatizer library from nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer() 

horoscope_lemmatize = []
for riga in df.index:
    horoscope_lemmatize_tmp= []
    for i in range(len(df['horoscope_token'][riga])):
        tmp1 = lemmatizer.lemmatize(df['horoscope_token'][riga][i])
        horoscope_lemmatize_tmp.append(tmp1)
    horoscope_lemmatize.append(horoscope_lemmatize_tmp)

df['horoscope_lemmatize_nltk'] = horoscope_lemmatize
df.head()


In [ ]:

nlp = spacy.load("en_core_web_sm")
# Lemmatization
def lemmatization(horoscopes, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    horoscope_out = []
    for sent in horoscopes:
        doc = nlp(' '.join(sent))
        horoscope_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return horoscope_out

df['horoscope_lemmatize'] = pd.Series(lemmatization(df['horoscope_token'], allowed_postags=['NOUN', 'ADJ']))
df.head()

Stop Words

In [ ]:
df['tokens_no_stop_stem'] = remove_stopwords(df['horoscope_stemming'])
df['tokens_no_stop_lem'] = remove_stopwords(df['horoscope_lemmatize'])

In [ ]:
df.reset_index(drop = True, inplace=True)

Part of speech tagging (POS)

In [ ]:
#nltk.download("all")

In [ ]:
#horoscope_pos = []
#for riga in df.index:
#    horoscope_pos_tmp= []
#    for i in range(len(df['horoscope_token'][riga])):
#        tmp1 = nltk.pos_tag(df['horoscope_token'][i])
#        horoscope_pos_tmp.append(tmp1)
#    horoscope_pos.append(horoscope_pos_tmp)

#df['horoscope_pos'] = horoscope_pos

Word CLoud

In [ ]:
# WordCloud

def rejoin_words(row):
    words = row['tokens_no_stop_lem']
    joined_words = (' '.join(words))
    return joined_words

In [ ]:
df['no_stop_joined'] = df.apply(rejoin_words, axis = 1)

In [ ]:
all_words = ' '.join([text for text in df['no_stop_joined']])

In [ ]:
wordcloud = WordCloud(width=900, height=600, random_state=21, max_font_size=110, background_color='ghostwhite', max_words=200, colormap='Dark2').generate(all_words)

plt.figure(figsize=(12, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

Cleaning effects - Length of horoscopes

In [ ]:
df['horoscope_length'] = df['Horoscope'].str.len()
df['cleaned_horoscope_length'] = df['no_stop_joined'].str.len()
df_lengths = df[['horoscope_length', 'cleaned_horoscope_length']]

In [ ]:
x1 = df_lengths.horoscope_length
x2 = df_lengths.cleaned_horoscope_length
plt.figure(figsize=(15, 6))
plt.suptitle('Length of horoscope as number of characters', fontsize = 14, fontweight = 'bold')
plt.subplot(1, 2, 1)
sns.distplot(x1, color='black', label='No. characters', bins=35, hist_kws={'alpha':0.5, 'rwidth':0.8})
plt.title('Original Horoscope', fontsize=12, fontweight='bold')
plt.xlabel('Number of characters', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.xlim([0, 400])

# Chart 2: Derivative Function
plt.subplot(1, 2, 2)
sns.distplot(x2, color='black', label='No. characters', bins=17, hist_kws={'alpha':0.5, 'rwidth':0.8})
plt.title('Cleaned horoscopes', fontsize=12, fontweight='bold')
plt.xlabel('Number of horoscopes', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.xlim([0, 400])

plt.show()

In [ ]:
df['horoscope_words'] = df['Horoscope'].str.split().str.len()
df['cleaned_horoscope_words'] = df['no_stop_joined'].str.split().str.len()
df_lengths = df[['horoscope_words', 'cleaned_horoscope_words']]

In [ ]:
x_1 = df_lengths.horoscope_words
x_2 = df_lengths.cleaned_horoscope_words
plt.figure(figsize = (15, 6))
plt.suptitle('Length of horoscope as number of words.', fontsize=14, fontweight='bold')
plt.subplot(1, 2, 1)

sns.distplot(x_1, color='black', label='No. Words', bins=25, hist_kws={'alpha':0.5, 'rwidth':0.8})
plt.title('Original Horoscopes', fontsize=12, fontweight='bold')
plt.xlabel('Number of words', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.xlim([0, 70])

# Chart 2: Derivative Function
plt.subplot(1, 2, 2)
sns.distplot(x_2, color='black', label='No. Words', bins=15, hist_kws={'alpha':0.5, 'rwidth':0.8})
plt.title('Cleaned Horoscopes', fontsize=12, fontweight='bold')
plt.xlabel('Number of words', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.xlim([0, 70])

plt.show()

Top 25 most frequent words

In [ ]:
word_freq = pd.Series(np.concatenate([x.split() for x in df.no_stop_joined])).value_counts()

In [ ]:
word_df = pd.Series.to_frame(word_freq)
word_df['word'] = list(word_df.index)
word_df.reset_index(drop=True, inplace=True)
word_df.columns = ['freq', 'word']

In [ ]:
label = word_df['word'].head(25)
freq = word_df['freq'].head(25)
index = np.arange(len(freq))

print('Unique words:', len(word_df))
plt.figure(figsize=(12, 9))
plt.bar(index, freq, alpha=0.8, color='black')
plt.xlabel('Words', fontsize=13)
plt.ylabel('Frequency', fontsize=13)
plt.xticks(index, label, fontsize=11, rotation=90, fontweight='bold')
plt.title('Top 25 Words after preprocessing', fontsize=12, fontweight='bold')
plt.show()

In [ ]:
#   CUSTOM STOPWORDS

stop_words.extend(['aries', 'taurus', 'gemini', 'cancer', 'leo', 'virgo', 'libra', 'scorpio', 'saggitarius','sagittarius', 'capricorn', 'acquarius','pisces', 'aquarius', 'today', 'could', 'might', 'may', 'day', 'thing', 'likely', 'perhaps', 'probably'])

df['tokens_no_stop_stem'] = remove_stopwords(df['horoscope_stemming'])
df['tokens_no_stop_lem'] = remove_stopwords(df['horoscope_lemmatize'])

In [ ]:
# WordCloud

def rejoin_words(row):
    words = row['tokens_no_stop_lem']
    joined_words = (' '.join(words))
    return joined_words

In [ ]:
df['no_stop_joined'] = df.apply(rejoin_words, axis = 1)

In [ ]:
all_words = ' '.join([text for text in df['no_stop_joined']])

In [ ]:
wordcloud = WordCloud(width=900, height=600, random_state=21, max_font_size=110, background_color='ghostwhite', max_words=200, colormap='Dark2').generate(all_words)

plt.figure(figsize=(12, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
word_freq = pd.Series(np.concatenate([x.split() for x in df.no_stop_joined])).value_counts()

In [ ]:
word_df = pd.Series.to_frame(word_freq)
word_df['word'] = list(word_df.index)
word_df.reset_index(drop=True, inplace=True)
word_df.columns = ['freq', 'word']

In [ ]:
label = word_df['word'].head(25)
freq = word_df['freq'].head(25)
index = np.arange(len(freq))

print('Unique words:', len(word_df))
plt.figure(figsize=(12, 9))
plt.bar(index, freq, alpha=0.8, color='black')
plt.xlabel('Words', fontsize=13)
plt.ylabel('Frequency', fontsize=13)
plt.xticks(index, label, fontsize=11, rotation=90, fontweight='bold')
plt.title('Top 25 Words after preprocessing (NOUNS and ADJ)', fontsize=12, fontweight='bold')
plt.show()

Sentiment Analysis

In [ ]:
# nltk sentiment analysis
#from nltk.sentiment.sentiment_analyzer import SentimentAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()
for sentence in df.index:
  ss = sid.polarity_scores(df['Horoscope'][sentence])
  for k in sorted(ss):
    print('{0}: {1}, '.format(k, ss[k]), end='')
  print()


In [ ]:
# applying to all sentences
def sid_compound(text):
  sid = SentimentIntensityAnalyzer()
  ss = sid.polarity_scores(text)
  return(ss['compound'])

def sid_pos(text):
  sid = SentimentIntensityAnalyzer()
  ss = sid.polarity_scores(text)
  return(ss['pos'])

def sid_neg(text):
  sid = SentimentIntensityAnalyzer()
  ss = sid.polarity_scores(text)
  return(ss['neg'])

def sid_neu(text):
  sid = SentimentIntensityAnalyzer()
  ss = sid.polarity_scores(text)
  return(ss['neu'])


df['sid_score'] = df['Horoscope'].apply(sid_compound)
df['sid_pos'] = df['Horoscope'].apply(sid_pos)
df['sid_neg'] = df['Horoscope'].apply(sid_neg)
df['sid_neu'] = df['Horoscope'].apply(sid_neu)

df.head()

In [ ]:
print('positive:', df.sid_pos.mean())

In [ ]:
print('negative:', df.sid_neg.mean())

In [ ]:
print('neutre:', df.sid_neu.mean())

In [ ]:
# textblob sentiment analysis
from textblob import TextBlob
def detect_polarity(text):
    return TextBlob(text).sentiment

for sentence in df.index:
    print(df['Horoscope'][sentence])
    print(detect_polarity(df['Horoscope'][sentence]))

In [ ]:
# applying to all sentences
def detect_polarity2(text):
    return TextBlob(text).sentiment.polarity

def detect_subjectivity(text):
    return TextBlob(text).sentiment.subjectivity

df['sid_score'] = df['Horoscope'].apply(sid_compound)
df['tb_score'] = df['Horoscope'].apply(detect_polarity2)
df['tb_score_detect_subjectivity'] = df['Horoscope'].apply(detect_subjectivity)

df.head()

In [ ]:
# graphing results
import seaborn as sns
import matplotlib.pyplot as plt

plt.title("TextBlob vs. Vader")
plt.xlabel("TextBlob - Compound score")
plt.ylabel("Vader - Compound score")
plt.axhline(color='black', linestyle='-')
plt.axvline(color='black', linestyle='-')
plt.rcParams['figure.figsize'] = [15,10] #altezza,lunghezza
plt.xlim(-1,1)
plt.scatter(x=df['tb_score'], y=df['sid_score'], marker="s")


In [ ]:
plt.hist(x=df['sid_score'], alpha=0.5) 

plt.legend()
plt.xlim(-1,1)
plt.title('Compound score (VADER)')

In [ ]:
plt.hist(x=df['tb_score_detect_subjectivity'], alpha=0.5, label='Subjectivity') 

plt.legend()
plt.title('Subjectivity')

In [ ]:

plt.hist(x=df['tb_score'], alpha=0.5) 
plt.legend()
plt.xlim(-1,1)
plt.title('Compound score (TextBlob)')